In [1]:
# Uninstall the current version of scikit-learn
!pip uninstall -y scikit-learn

# Install scikit-learn version 1.3.2
!pip install scikit-learn==1.3.2

# Verify the installation
import sklearn
print(sklearn.__version__)

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 116.4 MB/s eta 0:00:00
1.3.2


In [2]:
from sklearn.decomposition import PCA
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import math
import torch.nn as nn

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

class CTDataset_all(Dataset):
    def __init__(self, filepath):
        self.flatten = nn.Flatten()
        self.x, self.y = torch.load(filepath, weights_only=False)
        self.x = self.x / 255.
        self.x = self.flatten(self.x)
        self.y = F.one_hot(self.y, num_classes=10).to(float)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

torch.manual_seed(42)
train_ds = CTDataset_all('./training.pt')
test_ds = CTDataset_all('./test.pt')

torch.manual_seed(42)
train_AE_set, train_cond_gen_set = torch.utils.data.random_split(train_ds, [30000, 30000])

x_train, y_train = train_cond_gen_set[:]
x_test, y_test = test_ds[:]

x_train, y_train = x_train.cpu().detach().numpy(), y_train.cpu().detach().numpy()
x_test, y_test = x_test.cpu().detach().numpy(), y_test.cpu().detach().numpy()

classifiers = []
param_dist = {
    "n_neighbors":
    [i for i in range(1, int(math.sqrt(x_train.shape[0])))]
}
random_search = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_distributions=param_dist,
    n_iter=60,
    cv=5,
    n_jobs=-1,
    random_state=42 )
classifiers.append(random_search.fit(x_train, y_train))

test_accuracy = classifiers[0].score(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.962
